# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [1]:
library(dplyr)
require(tidyr)
library(ozmaps)
library(galah)
library(data.tree)
library(sf)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr


Attaching package: ‘galah’


The following object is masked from ‘package:tidyr’:

    unnest


The following object is masked from ‘package:dplyr’:

    desc


The following object is masked from ‘package:stats’:

    filter


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [2]:
here::i_am("RDS-output/Read-table-traits-per-species.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-db-exports



In [3]:
galah_config(email = readLines(here::here("secrets","galah-email.txt")))

### Read data

In [4]:
spp_traits_table <- readRDS(here::here("data", "output-Rdata","Summary-traits-species.rds"))


## Query taxonomy from Atlas of Living Australia

In [5]:
nsw <- st_transform(ozmaps::ozmap_states, 4326) |> filter(NAME == "New South Wales")

In [6]:
ALA_taxonomy_file <- here::here('data','ALA','NSW-plants-according-to-ALA.rds')
if (!file.exists(ALA_taxonomy_file)) {
    if (!dir.exists(here::here('data','ALA')))
        dir.create(here::here('data','ALA'))
    plant_families <- galah_call() |>
        galah_identify("plantae") |>
        galah_geolocate(nsw, type = "bbox")|>
        filter(rank == "family") |>
        atlas_taxonomy()
    saveRDS(file=ALA_taxonomy_file,plant_families)
} else {
    plant_families <- readRDS(ALA_taxonomy_file)
}

In [7]:
print(plant_families, pruneMethod = "dist", limit = 10)

# A tibble: 614 × 4
   name             rank     parent_taxon_concept_id            taxon_concept_id
   <chr>            <chr>    <chr>                              <chr>           
 1 Plantae          kingdom  NA                                 https://id.biod…
 2 Anthocerotophyta phylum   https://id.biodiversity.org.au/ta… https://id.biod…
 3 Anthocerotopsida class    https://id.biodiversity.org.au/no… https://id.biod…
 4 Anthocerotidae   subclass https://id.biodiversity.org.au/no… https://id.biod…
 5 Anthocerotales   order    https://id.biodiversity.org.au/no… https://id.biod…
 6 Anthocerotaceae  family   https://id.biodiversity.org.au/no… https://id.biod…
 7 Dendrocerotidae  subclass https://id.biodiversity.org.au/no… https://id.biod…
 8 Dendrocerotales  order    https://id.biodiversity.org.au/no… https://id.biod…
 9 Dendrocerotaceae family   https://id.biodiversity.org.au/no… https://id.biod…
10 Notothylatidae   subclass https://id.biodiversity.org.au/no… https://id.biod…
# ℹ 604 

In [8]:
plantae_id <- plant_families |> filter(name == "Plantae") |> pull(taxon_concept_id)

In [9]:
# Non vascular phyla
plant_families |> filter(parent_taxon_concept_id %in% plantae_id)


name,rank,parent_taxon_concept_id,taxon_concept_id
<chr>,<chr>,<chr>,<chr>
Anthocerotophyta,phylum,https://id.biodiversity.org.au/taxon/apni/51744352,https://id.biodiversity.org.au/node/ausmoss/9873243
Bryophyta,phylum,https://id.biodiversity.org.au/taxon/apni/51744352,https://id.biodiversity.org.au/taxon/ausmoss/10155928
Charophyta,phylum,https://id.biodiversity.org.au/taxon/apni/51744352,https://id.biodiversity.org.au/taxon/apni/51744351
Marchantiophyta,phylum,https://id.biodiversity.org.au/taxon/apni/51744352,https://id.biodiversity.org.au/taxon/ausmoss/51259285


In [10]:
fams <- plant_families |> 
    filter(rank == "family") |>
    select(family=name, family_concept_id=taxon_concept_id, order_concept_id=parent_taxon_concept_id)
fams <- fams |>
    left_join(plant_families, by=c("order_concept_id" = "taxon_concept_id")) |>
    select(family, family_concept_id, order_concept_id,tax_order=name,order_rank=rank,
          higher_concept_id=parent_taxon_concept_id)

In [11]:
fams <- fams |> left_join(plant_families, by=c("higher_concept_id" = "taxon_concept_id")) |>
    select(family, family_concept_id, order_concept_id,tax_order,order_rank,
          higher_concept_id,higher=name, higher_rank=rank)

In [12]:
table(fams$order_rank, fams$higher_rank)

            
             class order subclass superorder
  order          4     0       88        313
  subclass       2     0        0          0
  suborder       0    36        0          0
  superorder     0     0        3          0

In [13]:
fams <- fams |> 
    mutate(use_order = case_when(
        order_rank %in% "order" ~ tax_order,
        higher_rank %in% "order" ~ higher,
        TRUE ~ paste(order_rank, tax_order))
           )
    

In [14]:
glimpse(fams)

Rows: 446
Columns: 9
$ family            <chr> "Anthocerotaceae", "Dendrocerotaceae", "Notothyladac…
$ family_concept_id <chr> "https://id.biodiversity.org.au/node/ausmoss/8513200…
$ order_concept_id  <chr> "https://id.biodiversity.org.au/node/ausmoss/8513220…
$ tax_order         <chr> "Anthocerotales", "Dendrocerotales", "Notothyladales…
$ order_rank        <chr> "order", "order", "order", "order", "order", "order"…
$ higher_concept_id <chr> "https://id.biodiversity.org.au/node/ausmoss/8513221…
$ higher            <chr> "Anthocerotidae", "Dendrocerotidae", "Notothylatidae…
$ higher_rank       <chr> "subclass", "subclass", "subclass", "class", "supero…
$ use_order         <chr> "Anthocerotales", "Dendrocerotales", "Notothyladales…


In [15]:
fams |> filter(!order_rank %in% "order")

family,family_concept_id,order_concept_id,tax_order,order_rank,higher_concept_id,higher,higher_rank,use_order
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Distichiaceae,https://id.biodiversity.org.au/node/ausmoss/10071707,https://id.biodiversity.org.au/taxon/ausmoss/10155831,Dicranidae,subclass,https://id.biodiversity.org.au/taxon/ausmoss/10155927,Bryopsida,class,subclass Dicranidae
Boraginaceae,https://id.biodiversity.org.au/taxon/apni/51612493,https://id.biodiversity.org.au/taxon/apni/51744545,Asteranae,superorder,https://id.biodiversity.org.au/taxon/apni/51744349,Magnoliidae,subclass,superorder Asteranae
Icacinaceae,https://id.biodiversity.org.au/taxon/apni/51428890,https://id.biodiversity.org.au/taxon/apni/51744545,Asteranae,superorder,https://id.biodiversity.org.au/taxon/apni/51744349,Magnoliidae,subclass,superorder Asteranae
Dilleniaceae,https://id.biodiversity.org.au/taxon/apni/51434913,https://id.biodiversity.org.au/taxon/apni/51736237,Myrothamnanae,superorder,https://id.biodiversity.org.au/taxon/apni/51744349,Magnoliidae,subclass,superorder Myrothamnanae
Pteridaceae,https://id.biodiversity.org.au/taxon/apni/51449746,https://id.biodiversity.org.au/taxon/apni/51737882,Polypodiidae,subclass,https://id.biodiversity.org.au/taxon/apni/51744350,Equisetopsida,class,subclass Polypodiidae
Adelanthaceae,https://id.biodiversity.org.au/node/ausmoss/8303611,https://id.biodiversity.org.au/node/ausmoss/8303627,Cephaloziineae,suborder,https://id.biodiversity.org.au/node/ausmoss/8303630,Jungermanniales,order,Jungermanniales
Cephaloziellaceae,https://id.biodiversity.org.au/node/ausmoss/2898975,https://id.biodiversity.org.au/node/ausmoss/8303627,Cephaloziineae,suborder,https://id.biodiversity.org.au/node/ausmoss/8303630,Jungermanniales,order,Jungermanniales
Jamesoniellaceae,https://id.biodiversity.org.au/node/ausmoss/2911127,https://id.biodiversity.org.au/node/ausmoss/8303627,Cephaloziineae,suborder,https://id.biodiversity.org.au/node/ausmoss/8303630,Jungermanniales,order,Jungermanniales
Scapaniaceae,https://id.biodiversity.org.au/node/ausmoss/2892015,https://id.biodiversity.org.au/node/ausmoss/8303627,Cephaloziineae,suborder,https://id.biodiversity.org.au/node/ausmoss/8303630,Jungermanniales,order,Jungermanniales


### update table with taxonomy
Add column with the plant order from the taxonomy:

In [16]:
spp_traits_table$rank_order <- fams$use_order[match(spp_traits_table$family,fams$family)]

In [17]:
spp_traits_table |> 
    filter(is.na(rank_order)) |>
           group_by(family) |>
    summarise(spps = n_distinct(spp)) |>
    arrange(desc(spps)) |>
    head()

family,spps
<chr>,<int>
Fabaceae (Faboideae),923
Fabaceae (Mimosoideae),452
Fabaceae (Caesalpinioideae),140
Malaceae,61
Lomandraceae,47
Anthericaceae,37


In [18]:
fabales <- c("Fabaceae (Faboideae)", "Fabaceae (Mimosoideae)", "Fabaceae (Caesalpinioideae)" )
unplaced <- c("Dilleniaceae", "Flacourtiaceae")
single_family <- c("Boraginaceae")
rosales <- c("Malaceae")
lamiales <- c("Myoporaceae", "Buddlejaceae")
polypodiales <- c("Pteridaceae")
asparagales <- c("Anthericaceae","Phormiaceae","Lomandraceae", "Agavaceae", "Hyacinthaceae")
ranunculales <- c("Fumariaceae")
malvales <- c("Tiliaceae")


In [19]:
spp_traits_table <- spp_traits_table %>% mutate(
    rank_order = case_when(
            family %in% fabales ~ "Fabales",
            family %in% unplaced ~ "unplaced",
            family %in% rosales ~ "Rosales",
            family %in% lamiales ~ "Lamiales",
            family %in% polypodiales ~ "Polypodiales",
            family %in% asparagales ~ "Asparagales",
            family %in% ranunculales ~ "Ranunculales",
            family %in% malvales ~ "Malvales",
            family %in% single_family ~ sprintf("fam. %s",family),
            is.na(rank_order) ~ "unknown",
            TRUE ~ rank_order
        ))

In [20]:
spp_traits_table |> 
    filter(is.na(rank_order)) |>
           group_by(family) |>
    summarise(spps = n_distinct(spp)) |>
    arrange(desc(spps)) |>
    head()

family,spps
<chr>,<int>


In [21]:
spp_traits_table |> 
    filter(rank_order %in% c(NA,"unknown")) |>
           group_by(family) |>
    summarise(spps = n_distinct(spp)) |>
    arrange(desc(spps)) |>
    head()

family,spps
<chr>,<int>
Unknown Flora,32
Athyriaceae,15
Viscaceae,13
Stackhousiaceae,12
Adoxaceae,10
Magnoliaceae,10


Need to check this again, according to David, there should be around 6000 plant species in NSW, is this list including that many synonyms?

In [22]:
table(spp_traits_table$current)


false  true 
 3627 12530 

In [23]:
spp_traits_table %>%
    #filter(current %in% "true") %>%
    group_by(establishment,taxonrank %in% "Species") %>%
    summarise(total=n(), nspp = n_distinct(spp), ncurrent=n_distinct(current_species), .groups = "drop") 

establishment,"taxonrank %in% ""Species""",total,nspp,ncurrent
<chr>,<lgl>,<int>,<int>,<int>
"Alive in NSW, Native",FALSE,1813,1813,1251
"Alive in NSW, Native",TRUE,10556,10556,8169
"Extinct in NSW, Native",FALSE,9,9,4
"Extinct in NSW, Native",TRUE,38,38,28
Hybrid,FALSE,1,1,1
Introduced,FALSE,399,399,242
Introduced,TRUE,3309,3309,2804
Not Known from NSW,FALSE,5,5,5
Not Known from NSW,TRUE,27,27,26


Examine records for species that are not current:

In [24]:
head(subset(spp_traits_table, current %in% "false"))

,family,genus,spp,scientific_name,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1,rank_order
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
7,Poaceae,Anthosachne,1.167971e-320,Elymus scaber,1.133979e-319,Anthosachne scabra,Species,"Alive in NSW, Native",false,4.940656e-324,⋯,0,0,0,0,0.000000e+00,4.940656e-324,0,4.940656e-324,1.482197e-323,Poales
12,Ericaceae,Agiortia,1.170442e-320,Leucopogon pedicillatus,8.518186e-320,Agiortia pedicellata,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Ericales
14,Fabaceae (Faboideae),Abrus,1.171430e-320,Abrus precatorius,6.625420e-320,Abrus precatorius subsp. precatorius,Subspecies,Introduced,false,0.000000e+00,⋯,0,0,0,0,9.881313e-324,0.000000e+00,0,0.000000e+00,4.940656e-324,Fabales
15,Poaceae,Paractaenum,1.171924e-320,Paractaenum novae-hollandiae subsp. reversum,1.732688e-320,Paractaenum novae-hollandiae,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Poales
17,Pteridaceae,Adiantum,1.172912e-320,Adiantum silvaticum var. silvaticum,3.738595e-320,Adiantum silvaticum,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Polypodiales
28,Plantaginaceae,Linaria,1.178347e-320,Linaria genistifolia,6.265741e-320,Linaria dalmatica,Species,Introduced,false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Lamiales


In [25]:
spp_traits_table %>% slice(1:5)

family,genus,spp,scientific_name,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1,rank_order
<chr>,<chr>,<int64>,<chr>,<int64>,<chr>,<chr>,<chr>,<chr>,<int64>,⋯,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<chr>
Brassicaceae,Lepidium,1.165007e-320,Lepidium oxytrichum,1.165007e-320,Lepidium oxytrichum,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,2.470328e-323,0,0.000000e+00,0,1.482197e-323,Brassicales
Myrtaceae,Eucalyptus,1.165501e-320,Eucalyptus williamsiana,1.165501e-320,Eucalyptus williamsiana,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0,2.964394e-323,Myrtales
Myrtaceae,Melaleuca,1.165995e-320,Melaleuca glomerata,1.165995e-320,Melaleuca glomerata,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,9.881313e-324,0,0.000000e+00,0,2.964394e-323,Myrtales
Apiaceae,Actinotus,1.166489e-320,Actinotus helianthi,1.166489e-320,Actinotus helianthi,Species,"Alive in NSW, Native",true,0,⋯,0,9.881313e-324,0,4.940656e-324,1.482197e-323,0,1.482197e-323,0,7.905050e-323,Apiales
Apiaceae,Apium,1.166983e-320,Apium prostratum,1.166983e-320,Apium prostratum,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,1.976263e-323,0,0.000000e+00,0,4.940656e-324,Apiales


## Literature vs. field work data
Let's check how many valid species have information from both sources


In [26]:
 spp_traits_table |> 
    filter(
        #current %in% "true", 
           taxonrank %in% "Species",
           establishment %in% "Alive in NSW, Native") |>
    mutate(
     fieldwork_sources =  nquadrat>0,
     literature_sources = (disp1 +
                           germ1 + germ8 + 
                           rect2 + 
                           grow1 + 
                           repr2 + repr3a + repr3 + repr4 +  
                           surv1 + surv4 + surv5 + surv6 + surv7) > 0
    ) |> 
    group_by(fieldwork_sources, literature_sources) |>
    summarise(total = n_distinct(scientific_name), total_current = n_distinct(current_species), .groups = "drop")

fieldwork_sources,literature_sources,total,total_current
<lgl>,<lgl>,<int>,<int>
FALSE,FALSE,4574,3844
FALSE,TRUE,5129,4843
TRUE,FALSE,87,87
TRUE,TRUE,724,723


In [27]:
4843+87+723

[1] 5653

In [28]:
sum(spp_traits_table$repr2>0)

[1] 218

In [29]:
saveRDS(file=here::here('data', 'output-Rdata', 'Summary-traits-species-orders.rds'),spp_traits_table)